# Assignment 2 - Search

In this assignment, you are going to complete the implementation of a state representation for the Traveling Salesman Problem and perform several simulations of simulated annealing.

What you need to do:
<br>1. Follow the instructions and complete the parts with **# TODO**.
<br>2. Complete the implementations. 
<br>3. Run experiments to search for the best setting of parameters **k** and **lam**.
<br>4. Report the results using tables.
<br>5. Discuss your findings.

# Your Information

# Implementations

**TODO**: Complete the implementation of `TSPNode` and `read_TSP_from_file`. See below.

In [ ]:
from matplotlib import pylab
import matplotlib.pyplot as plt
import numpy as np
import string
from collections import defaultdict

In [ ]:
class Node:
    def __init__(self, state, parent = None):
        self.state = state
        self.parent = parent

    def __repr__(self):
        return "Node: {}".format(self.state)

    def path(self):
        current = self
        path_back = [current]
        while current.parent is not None:
            path_back.append(current.parent)
            current = current.parent
        return reversed(path_back)

    def expand(self):
        raise NotImplementedError

    def value(self):
        raise NotImplementedError

In [ ]:
class TSPNode(Node):
    _random_state = None
    _distances = None

    def __init__(self, state, parent = None):
        """
        A state is an ordered list of cities. For e.g., ["A", "C", "D", "B"].
        This represents the solution of A - C - D - B - A.
        """        
        super().__init__(state, parent)
    
    def __repr__(self):
        return "Node: <{}> {:.1f}".format(" ".join(self.state), self.value())
    
    def expand(self):
        """
        Generate one random neighbor using the TSPNode._random_state.
        
        The random neighbor should be generated as follows. Pick two cities at random, and swap them.
        
        return:

        [neighbor_node]: a list of one TSPNode whose parent is this node.
        """
        neigbhor_node = None #TODO Implement.
        
        return [neigbhor_node]
    
    def value(self):
        """
        Calculate the total cost.
        
        return:
        -1*total_distance: the total cost of current state
        
        """
        total_distance = 0
        n = len(self.state)

        for i in range(n):
            from_c = self.state[i]
            if i == n-1:
                to_c = self.state[0]
            else:
                to_c = self.state[i+1]
            
            total_distance += TSPNode._distances[from_c][to_c]

        return -1*total_distance

In [ ]:
class Graph:
    def __init__(self):
        self.distances = defaultdict(dict)

    def add_edge(self, from_c, to_c, dist):
        self.distances[from_c][to_c] = dist
        self.distances[to_c][from_c] = dist

In [ ]:
def make_graph(city_coords, dist_mat):
    """
    Create a graph for the given TSP
    
    param:
    city_coords: dictionary of cities
    dist_mat: distance matrix
    
    return:
    graph: an instance of Graph class that saved all necessary edges and costs
    
    """
    graph = Graph()

    cities = list(city_coords.keys())

    for i in range(len(cities)-1):
        from_c = cities[i]
        for j in range(i+1, len(cities)):
            to_c = cities[j]
            dist = dist_mat[i][j]
            graph.add_edge(from_c, to_c, dist)
    return graph

In [ ]:
def init_state(seed, city_coords):
    """
    Create an initial state node 
    
    param:
    seed: random seed
    city_coords: dictionary of cities
    
    return:
    initial_state: an instance of TSPNode class with a randomly generated state
    
    """
    rand_state = np.random.RandomState(seed=seed)
    
    cities = list(city_coords.keys())
    shuffle_cities = list(rand_state.permutation(cities))
    initial_state = TSPNode(shuffle_cities)
    return initial_state

In [ ]:
def exp_schedule(k, lam):
    """
    The exponential schedule function for simulated annealing
    
    param:
    k: initial temperature
    lam: cooling factor lam
    
    return:
    a function that accepts the current number of iteration as input and outputs a new temperature
    
    """
    return lambda t: k * np.exp(-lam * t) 

def linear_schedule(k, lam):
    return lambda t: max(0, k - lam*t)

def log_schedule(k, lam):
    return lambda t: k / (1+lam*np.log(t+1))

In [ ]:
def simulated_annealing(initial_n, temp_schedule, max_iter, random_state):
    """
    Simulated annealing algorithm
    
    param:
    initial_n: initial state
    temp_schedule: temperature schedule function
    max_iter: the max number of iterations
    random_state: random state used to select random node or generate probability
    
    return:
    current_n: a instance of TSPNode as solution state
    
    """
    current_n = initial_n
    for t in range(max_iter):

        T = temp_schedule(t)
        next_nodes = current_n.expand()

        if len(next_nodes) == 0:
            return current_n
        else:
            next_n = random_state.choice(next_nodes)

            delta_e =  next_n.value() - current_n.value()

            if delta_e > 0:
                current_n = next_n
            else:
                p = np.exp(delta_e/T)
                #print("{:.1f} -> {:.1f}: {:.3f}".format(current_n.state, next_n.state, p))
                if random_state.random() < p:
                    current_n = next_n
    return current_n

In [ ]:
def check_margin(x, y, x_inner_lim, y_inner_lim):
    if x < x_inner_lim[0]:
        return True
    elif x > x_inner_lim[1]:
        return True
    elif x_inner_lim[0] <= x <= x_inner_lim[1] and (y > y_inner_lim[1] or y < y_inner_lim[0]):
        return True
    else:
        return False

In [ ]:
def TSP_generator(seed, x_inner_lim, x_outer_lim, y_inner_lim, y_outer_lim, num_city):
    i = 0
    cities = set()
    dist_mat = np.zeros((num_city, num_city))

    # Generate cities
    while len(cities) < num_city:
        rand_state = np.random.RandomState(seed=seed + i)
        x_coord = rand_state.uniform(x_outer_lim[0], x_outer_lim[1], num_city)
        y_coord = rand_state.uniform(y_outer_lim[0], y_outer_lim[1], num_city)

        # Check if the generated coordinates are in the inner area
        new_set = [(x, y) for x, y in zip(x_coord, y_coord) if check_margin(x, y, x_inner_lim, y_inner_lim)]
        cities.update(new_set)
        i += 1

    cities = list(cities)[:num_city]
    cities_dict = dict(zip(string.ascii_uppercase, cities))

    # Generate edge cost
    coordinates = np.asarray(cities)
    for i in range(num_city):
        for j in range(i + 1, num_city):
            dist_mat[i][j] = np.sqrt(np.sum((coordinates[i] - coordinates[j]) ** 2))
    return cities_dict, dist_mat

In [8]:
import numpy as np
dist_mat = np.array([[   0.        , 2306.60634444, 2245.72708178, 1578.65864374,
        1937.09608407, 1974.62369488, 2363.89365112, 1925.40149451,
          65.97362736, 2230.75717826],
       [   0.        ,    0.        ,  558.23991702, 1549.95321072,
        1077.82824388, 1025.18964386,   68.02865876, 1252.10120757,
        2246.54308272,  122.44166775],
       [   0.        ,    0.        ,    0.        , 1890.287398  ,
         585.78190259,  525.29502333,  609.24246003,  753.27527396,
        2180.80485229,  639.89776843],
       [   0.        ,    0.        ,    0.        ,    0.        ,
        2031.3129121 , 2021.19671303, 1567.71551366, 2148.34146724,
        1553.12094904, 1428.65853806],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,   61.9646973 , 1140.48734801,  174.28905134,
        1871.25856795, 1120.19716264],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , 1087.03727617,  229.99682626,
        1908.69916343, 1071.32949403],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        , 1314.76993321,
        2304.2681851 ,  146.54155391],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
        1860.22799719, 1293.4148353 ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        , 2171.92743052],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ]])
dist_mat = [[np.round(d) for d in sub_d] for sub_d in dist_mat]
dist_mat

[[0.0, 2307.0, 2246.0, 1579.0, 1937.0, 1975.0, 2364.0, 1925.0, 66.0, 2231.0],
 [0.0, 0.0, 558.0, 1550.0, 1078.0, 1025.0, 68.0, 1252.0, 2247.0, 122.0],
 [0.0, 0.0, 0.0, 1890.0, 586.0, 525.0, 609.0, 753.0, 2181.0, 640.0],
 [0.0, 0.0, 0.0, 0.0, 2031.0, 2021.0, 1568.0, 2148.0, 1553.0, 1429.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 62.0, 1140.0, 174.0, 1871.0, 1120.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1087.0, 230.0, 1909.0, 1071.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1315.0, 2304.0, 147.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1860.0, 1293.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2172.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [9]:
num_city= 10
for i in range(num_city):
    for j in range(i + 1, num_city):
        print(dist_mat[i][j])

2307.0
2246.0
1579.0
1937.0
1975.0
2364.0
1925.0
66.0
2231.0
558.0
1550.0
1078.0
1025.0
68.0
1252.0
2247.0
122.0
1890.0
586.0
525.0
609.0
753.0
2181.0
640.0
2031.0
2021.0
1568.0
2148.0
1553.0
1429.0
62.0
1140.0
174.0
1871.0
1120.0
1087.0
230.0
1909.0
1071.0
1315.0
2304.0
147.0
1860.0
1293.0
2172.0


In [ ]:
def TSP_plot(city_coords):
    pylab.rcParams['figure.figsize'] = (10.0, 8.0)

    for k_name, v_coord in city_coords.items():
        x, y = v_coord
        plt.scatter(x, y, marker='x', c='r', s=100)
        plt.text(x, y + 0.04, k_name, fontsize='xx-large')
    plt.show()

In [ ]:
def solution_visualization(solution, city_coords):
    """
    Visualize the final solution
    
    param:
    solution: a TSPNode of final state
    city_coords: dictionary of cities
    
    """
    pylab.rcParams['figure.figsize'] = (10.0, 8.0)
    
    for k_name, v_coord in city_coords.items():
        x, y = v_coord
        plt.scatter(x, y, marker='x', c='r', s=100)
        plt.text(x, y+0.04, k_name, fontsize='xx-large')
        
    # Draw the line between two cities
    for i, c in enumerate(solution.state):
        x_start, y_start = city_coords[c]
        if i != len(solution.state) - 1:
            x_end, y_end = city_coords[solution.state[i+1]]
        else:
            x_end, y_end = city_coords[solution.state[0]]
        
        x, y = [x_start, x_end], [y_start, y_end]
        x_mid, y_mid = (x_start + x_end)/2, (y_start + y_end)/2
        plt.plot(x, y, 'ro-')

    plt.show()

In [ ]:
def read_TSP_from_file():
    """
    Read cities from file
    
    return:
    city_coords: a Dictionary as {city_name: (x_coordinate, y_coordinate)}
    dist_mat: a matrix of euclidean distance between each pair of cities
    
    """
    # TODO

    city_coords = None
    dist_mat = None
    
    return city_coords, dist_mat

# Testing your implementation

## Generate or read a TSP problem

In [ ]:
rand_seed = 11
rand_state = np.random.RandomState(rand_seed)

In [ ]:
x_inner_range, x_outer_range = (-99, 99), (-100, 100)
y_inner_range, y_outer_range = (-99, 99), (-100, 100)

num_city = 10
max_iter = 1000
schedule_k = 500
schedule_lam = 0.25

In [ ]:
# Option 1: read cities from a file
#cities_coords, dist_mat = read_TSP_from_file()
# Option 2: randomly generate cities from a TSP generator
cities_coords, dist_mat = TSP_generator(rand_seed, x_inner_range, x_outer_range, y_inner_range, y_outer_range, num_city)

In [ ]:
cities_coords

In [ ]:
dist_mat

In [ ]:
TSP_plot(cities_coords)

In [ ]:
tsp_graph = make_graph(cities_coords, dist_mat)
TSPNode._distances = tsp_graph.distances
TSPNode._random_state = rand_state

## 2. Generate initial state

In [ ]:
initial_n = init_state(rand_seed, cities_coords)
initial_n.state

## 3. Run Simulated Annealing

In [ ]:
t_schedule = exp_schedule(k=schedule_k, lam=schedule_lam)

In [ ]:
solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
solution_n.state

In [ ]:
sol_path = list(solution_n.path())
for node in sol_path:
    print(node)

## 4. Visualize final solution

In [ ]:
solution_visualization(solution_n, cities_coords)

# Run Simulations

## TSP-1 (Large cost)

The TSP problem generated with large costs. Please use the given random seed. 

In [ ]:
num_city = 10
x_inner_range, x_outer_range = (-900, 900), (-1000, 1000)
y_inner_range, y_outer_range = (-900, 900), (-1000, 1000)

cities_coords, dist_mat = TSP_generator(1234, x_inner_range, x_outer_range, y_inner_range, y_outer_range, num_city)
tsp_graph = make_graph(cities_coords, dist_mat)
TSPNode._distances = tsp_graph.distances

### Brute Force

In [ ]:
# TODO Implement brute force search and record the optimal result and visualize it

### Exponential Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, 1234

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = exp_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

### Linear Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, 1234

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = linear_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

### Log Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, 1234

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = log_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

In [ ]:
# TODO: Present a table of your results for TSP-1. Consider using pandas.

## TSP-2 (Small cost)

The TSP problem generated with small costs. Please use your CWID as the random seed. 

In [ ]:
num_city = 10
x_inner_range, x_outer_range = (-9, 9), (-15, 15)
y_inner_range, y_outer_range = (-9, 9), (-15, 15)

# TODO: Please replace "4321" with your own CWID number after "A"
your_own_seed = int("4321")

cities_coords, dist_mat = TSP_generator(your_own_seed, x_inner_range, x_outer_range, y_inner_range, y_outer_range, num_city)
tsp_graph = make_graph(cities_coords, dist_mat)
TSPNode._distances = tsp_graph.distances

### Brute Force

In [ ]:
# TODO

### Exponential Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, your_own_seed

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = exp_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

### Linear Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, your_own_seed

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = linear_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

### Log Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, your_own_seed

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = log_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

In [ ]:
# TODO: Present a table of your results for TSP-2. Consider using pandas.

## TSP-3 (berlin52)

The TSP problem loaded from the file "berlin52.tsp" with 52 cities.

In [ ]:
cities_coords, dist_mat = read_TSP_from_file()
tsp_graph = make_graph(cities_coords, dist_mat)
TSPNode._distances = tsp_graph.distances

### Exponential Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, 1234

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = exp_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

### Linear Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, 1234

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = linear_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

### Log Schedule

In [ ]:
max_iter, num_trials, rand_seed = 500, 10, 1234

# TODO: you need to decide the value set of k and lam by yourself
k_set = [10, 100, 500]
lam_set = [0.001, 0.01, 0.25, 0.5, 0.99]

res_dict = {}
for k in k_set:
    for lam in lam_set:
        t_schedule = log_schedule(k=k, lam=lam)
        
        cost_list = []
        for trial_idx in range(num_trials):
            rand_state = np.random.RandomState(rand_seed+trial_idx)
            TSPNode._random_state = rand_state
            initial_n = init_state(rand_seed+trial_idx, cities_coords)
            
            solution_n = simulated_annealing(initial_n, t_schedule, max_iter, rand_state)
            cost_list.append(solution_n.value())
        
        res_dict[(k, lam)] = np.average(cost_list)

In [ ]:
res_dict

In [ ]:
# TODO: Present a table of your results for TSP-3. Consider using pandas.

# Report

**TODO** Discuss your findings.